<a href="https://colab.research.google.com/github/SwatiGupta1608/Fake-News-Detection/blob/main/dynamicrating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer

In [ ]:
# Load data into a pandas dataframe
data = pd.read_csv('company_ratings.csv')

In [ ]:
# # Identify invalid date rows
# invalid_date_rows = pd.to_datetime(data['Date of Payment Given'], errors='coerce').isna()
# # Print rows with invalid dates
# print(data[invalid_date_rows])


In [ ]:
# Preprocess data
data['Date of Payment Given'] = pd.to_datetime(data['Date of Payment Given'])
data['Actual Date of Payment'] = pd.to_datetime(data['Actual Date of Payment'])
data['Days Delayed'] = (data['Actual Date of Payment'] - data['Date of Payment Given']).dt.days
data.drop(columns=['Date of Payment Given', 'Actual Date of Payment'], inplace=True)

In [ ]:
# Split data into train and test sets
train = data.sample(frac=0.8, random_state=123)
test = data.drop(train.index)

In [ ]:
# Define features and target variable
target = 'Current Rating'
numeric_cols = train.select_dtypes(include=np.number).columns.tolist()
numeric_cols.remove(target)
features = numeric_cols


In [ ]:
# Drop rows with missing values
train.dropna(subset=[target], inplace=True)

In [ ]:
# Convert string values to numeric values
train[target] = pd.to_numeric(train[target], errors='coerce')

In [ ]:
# Replace missing values with mean value
imputer = SimpleImputer(strategy='mean')
train[features] = imputer.fit_transform(train[features])

In [ ]:
# Train random forest model
model = RandomForestRegressor(n_estimators=100, random_state=123)
model.fit(train[features], train[target])


RandomForestRegressor(random_state=123)

In [ ]:
# Predict on the test set and evaluate the model
test[target] = pd.to_numeric(test[target], errors='coerce')
test[features] = imputer.transform(test[features])
test.dropna(subset=[target], inplace=True)
predictions = model.predict(test[features])
mae = mean_absolute_error(test[target], predictions)
print("Mean absolute error:", mae)

Mean absolute error: 0.31397272727272707


In [ ]:
accuracy = (1 - mae/test[target].mean()) * 100
print("Accuracy:", accuracy, "%")

Accuracy: 91.98677494199536 %
